In [34]:
import bs4 as bs
import datetime as dt
import os
from pandas_datareader import data as pdr
import pickle
import requests
import fix_yahoo_finance as yf

yf.pdr_override()

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.replace('.', '-')
        ticker = ticker[:-1]
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers


# save_sp500_tickers()
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    start = dt.datetime(2015, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        print(ticker)
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = pdr.DataReader(ticker,'yahoo',start,end)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))


save_sp500_tickers()
get_data_from_yahoo()

MMM
AOS
ABT
ABBV
ABMD
ACN
ATVI
ADM
ADBE
ADP
AAP
AES
AFL
A
AIG
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AMD
AEE
AAL
AEP
AXP
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
ANTM
AON
APA
AAPL
AMAT
APTV
ANET
AIZ
T
ATO
ADSK
AZO
AVB
AVY
BKR
BLL
BAC
BBWI
BAX
BDX
WRB
BRK-B
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF-B
CHRW
CDNS
CZR
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CERN
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CTVA
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISCA
DISCK
DISH
DIS
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DRE
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ENPH
ETR
EOG
EPAM
EFX
EQIX
EQR
ESS
EL
ETSY
RE
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FAST
FRT
FDX
FITB
FRC
FE
FIS
FISV
FLT
FMC
F
FTNT
FTV
FBHS
FOXA
FOX
BEN
FCX
AJG
GRMN
IT
GE
GNRC
GD
GIS
GPC
GILD
GL
GPN
GM
GS
GWW
HAL
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HS

In [55]:
import pandas as pd
def compile_data():
    with open('sp500tickers.pickle','rb') as f:
        tickers = pickle.load(f)
    
    for i in range(len(tickers)):
        tickers[i] = tickers[i].replace('.', '-')
    
    main_df = pd.DataFrame()
    
    for count, ticker in enumerate(tickers):
        df = pd.read_csv(f'stock_dfs/{ticker}.csv')
        #df = df['Adj Close']
        df.set_index('Date',inplace=True)
        df.drop(['Close','Open','High','Low','Volume'],axis=1,inplace = True)
        df.rename(columns={'Adj Close':f'{ticker}_close'},inplace = True)
#         df.rename(columns = {'Adj Close':f'{ticker}_close',
#                             'Open':f'{ticker}_open',
#                             'High':f'{ticker}_high',
#                             'Low':f'{ticker}_low',
#                             'Volume':f'{ticker}_volume'},
#                   inplace = True)
              
        if main_df.empty:
            main_df = df
        else:
            main_df = pd.concat([main_df,df],axis =1)
            
        if count % 100 == 0:
            print(count)
            
    main_df.to_csv('sp500_data.csv')
compile_data()

0
100
200
300
400
500


In [61]:
data = pd.read_csv('sp500_data.csv')
data.head()

,Date,MMM_close,AOS_close,ABT_close,ABBV_close,ABMD_close,ACN_close,ATVI_close,ADM_close,ADBE_close,...,WMB_close,WTW_close,WYNN_close,XEL_close,XYL_close,YUM_close,ZBRA_close,ZBH_close,ZION_close,ZTS_close
0,2014-12-31,132.877563,25.342997,39.220860,47.978977,38.060001,79.011787,19.187967,42.031590,72.699997,...,28.958910,118.701988,131.287338,29.350044,34.679962,45.680447,77.410004,107.333237,25.172337,41.016884
1,2015-01-02,132.667328,25.163284,39.116322,48.308907,37.310001,78.596008,19.168921,42.047775,72.339996,...,29.139332,118.013245,129.636993,29.513472,34.689079,45.366924,77.430000,106.547775,24.978090,41.283794
2,2015-01-05,129.675308,24.723013,39.125023,47.399780,37.070000,77.268951,18.902290,40.592831,71.980003,...,27.985886,117.165565,127.430626,29.178459,32.530121,44.445164,76.339996,110.522377,24.042185,41.035954
3,2015-01-06,128.292496,24.520847,38.680733,47.165157,36.130001,76.711601,18.549955,39.792614,70.529999,...,27.605684,116.582779,124.712402,29.333715,32.338821,43.899632,75.790001,109.585533,23.123941,40.635601
4,2015-01-07,129.222458,24.700546,38.994358,49.071415,37.279999,78.321747,18.150003,40.390755,71.110001,...,27.457478,118.251656,128.374954,29.587006,32.593887,45.354382,77.720001,112.310951,23.344675,41.474419


In [62]:
data.shape

(1802, 506)

In [63]:
#detele the columns which have nan value
data = data.dropna(how='any',axis=1)
data.shape

(1802, 484)